In [41]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
%matplotlib inline

In [42]:
# read source and noise
sr, s = wavfile.read('dataset/sp01_001/audio/clean/sp01_001.wav')
sr, a = wavfile.read('source_noise/91_types_noises/128_n40.wav')
sr, c = wavfile.read('source_noise/14_car_IDL_35U_35D_55U_55D.wav')

In [43]:
s.shape[0], a.shape[0], c.shape[0]

(63147, 1280000, 3200000)

In [44]:
a = a[:s.shape[0] - 1920]
c = c[:s.shape[0] - 2303]

In [45]:
def mix_wav(source, ambient, car):
        snr_ambient = random.randint(-5, 10)
        snr_car = random.randint(-5, 10)
        
        source_len, ambient_len, car_len = source.shape[0], ambient.shape[0], car.shape[0]
        
        # Pad wave sequences
        if ambient_len > source_len:
            start_idx = random.randint(0, ambient_len - source_len)
            ambient = ambient[start_idx:start_idx + source_len]
        
        elif ambient_len < source_len:
            front = random.randint(0, source_len - ambient_len - 1)
            rear = source_len - ambient_len - front
            ambient = np.pad(ambient, (front, rear), 'wrap')
            
        if car_len > source_len:
            start_idx = random.randint(0, car_len - source_len)
            car = car[start_idx:start_idx + source_len]
        
        elif car_len < source_len:
            front = random.randint(0, source_len - ambient_len - 1)
            rear = source_len - car_len - front
            car = np.pad(car, (front, rear), 'wrap')
        
        assert source.shape[0] == ambient.shape[0] == car.shape[0]

        source_amp = sum(abs(source)) / len(source)

        ambient_amp = source_amp / (10 ** (snr_ambient / 20))
        car_amp = source_amp / (10 ** (snr_car / 20))
        ambient_norm = (ambient - ambient.mean()) / ambient.std()
        car_norm = (car - car.mean()) / car.std()

        ambient_scaled = ambient_amp * ambient_norm + ambient.mean()
        car_scaled = car_amp * car_norm + car.mean()

        mixed = source + ambient_scaled + car_scaled

        return mixed.astype(np.int16)

In [50]:
mixed = mix_wav(s, a, c)

In [51]:
wavfile.write('mixed.wav', sr, mixed)